In [1]:
#!pip install ocs-academic-hub
#!pip uninstall -y ocs_academic_hub
#!pip list

In [2]:
#!git clone https://github.com/cfoisy-osisoft/academic-hub.git

In [3]:
from ocs_academic_hub import OMFClient
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dateutil.parser import parse

C:\Users\zcheng\Anaconda3\lib\site-packages\typeguard\__init__.py:906: UserWarning: no type annotations present -- not typechecking ocs_academic_hub.omf_client.OMFClient.__init__
  warn('no type annotations present -- not typechecking {}'.format(function_name(func)))


# Data frame generation

In [4]:
start = pd.to_datetime('2021-03-20 19:32:00')
end = pd.to_datetime('2021-03-20 19:35:00')
freq = 20 #Hz
column = 4

In [5]:
time_range = pd.date_range(start, end, freq=f'{1/freq*1000000000}ns')

np.random.seed(seed=1111)
data = np.random.uniform(1, high=100, size=(len(time_range), column))

df = pd.DataFrame({'Timestamp': time_range}) 
#df = df.set_index('Timestamp')
i = 0
while i < column:
    df[f'col{i + 1}'] = data[:,i]
    i += 1
print(df)

                   Timestamp       col1       col2       col3       col4
0    2021-03-20 19:32:00.000  10.459371  92.575366  35.013769  31.737217
1    2021-03-20 19:32:00.050   1.198974  24.323878  24.541380  73.855671
2    2021-03-20 19:32:00.100  50.051340  78.658110  13.524125  61.058283
3    2021-03-20 19:32:00.150  47.145977  24.476080  44.080759  25.123480
4    2021-03-20 19:32:00.200  39.000151  84.000975  65.863288  15.696221
...                      ...        ...        ...        ...        ...
3596 2021-03-20 19:34:59.800  77.316408  98.087414  28.896870  71.278356
3597 2021-03-20 19:34:59.850  81.325958  57.351737  40.252543  24.371531
3598 2021-03-20 19:34:59.900   9.270223  86.376215  63.276650  40.157959
3599 2021-03-20 19:34:59.950  96.116558  35.603031  26.571837  67.697390
3600 2021-03-20 19:35:00.000  78.484158   6.750825  19.641894  31.234646

[3601 rows x 5 columns]


# Schedular 

In [6]:
from threading import Timer

class RepeatedTimer(object):
    def __init__(self, interval, function, *args, **kwargs):
        self._timer     = None
        self.interval   = interval
        self.function   = function
        self.args       = args
        self.kwargs     = kwargs
        self.is_running = False
        self.start()

    def _run(self):
        self.is_running = False
        self.start()
        self.function(*self.args, **self.kwargs)

    def start(self):
        if not self.is_running:
            self._timer = Timer(self.interval, self._run)
            self._timer.start()
            self.is_running = True

    def stop(self):
        self._timer.cancel()
        self.is_running = False

# Send OMF Message to EDS

In [7]:
from time import sleep

asset = 'test'
omf_client = OMFClient()

def event():
    df_rt=df[abs(df['Timestamp'] - pd.Timestamp.now())<=timedelta(seconds=1/freq)]
    omf_client.update_tags(df_rt, asset)
    #print(df_rt)

print ("starting...")
rt = RepeatedTimer(1/freq, event) # it auto-starts, no need of rt.start()
try:
    sleep((end-start).total_seconds()) # your long-running job goes here...
finally:
    rt.stop() # better in a try/finally block to make sure the program ends!
    print ("stopped!")

starting...
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-20 19:32:11.100000 to 2021-03-20 19:32:11.150000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-20 19:32:11.200000 to 2021-03-20 19:32:11.250000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-20 19:32:11.250000 to 2021-03-20 19:32:11.300000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-20 19:32:11.300000 to 2021-03-20 19:32:11.350000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-20 19:32:11.350000 to 2021-03-20 19:32:11.400000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-20 19:32:11.450000 to 2021-03-20 19:32:11.500000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-20 19:32:11.500000 to 2021-03-20 19:32:11.550000
>

>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-20 19:32:14.700000 to 2021-03-20 19:32:14.750000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-20 19:32:14.750000 to 2021-03-20 19:32:14.800000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-20 19:32:14.800000 to 2021-03-20 19:32:14.850000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-20 19:32:14.850000 to 2021-03-20 19:32:14.900000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-20 19:32:14.950000 to 2021-03-20 19:32:15
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-20 19:32:15 to 2021-03-20 19:32:15.050000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-20 19:32:15.050000 to 2021-03-20 19:32:15.100000
>> new tag(s): ['x.test.col

>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-20 19:32:18.300000 to 2021-03-20 19:32:18.350000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-20 19:32:18.400000 to 2021-03-20 19:32:18.450000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-20 19:32:18.450000 to 2021-03-20 19:32:18.500000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-20 19:32:18.500000 to 2021-03-20 19:32:18.550000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-20 19:32:18.550000 to 2021-03-20 19:32:18.600000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-20 19:32:18.650000 to 2021-03-20 19:32:18.700000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-20 19:32:18.700000 to 2021-03-20 19:32:18.750000
>> new tag(s)

>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-20 19:32:22.100000 to 2021-03-20 19:32:22.150000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-20 19:32:22.200000 to 2021-03-20 19:32:22.250000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-20 19:32:22.250000 to 2021-03-20 19:32:22.300000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-20 19:32:22.350000 to 2021-03-20 19:32:22.400000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-20 19:32:22.400000 to 2021-03-20 19:32:22.450000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-20 19:32:22.450000 to 2021-03-20 19:32:22.500000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-20 19:32:22.500000 to 2021-03-20 19:32:22.550000
>> new tag(s)

>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-20 19:32:26 to 2021-03-20 19:32:26.050000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-20 19:32:26.050000 to 2021-03-20 19:32:26.100000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-20 19:32:26.100000 to 2021-03-20 19:32:26.150000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-20 19:32:26.200000 to 2021-03-20 19:32:26.250000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-20 19:32:26.250000 to 2021-03-20 19:32:26.300000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-20 19:32:26.300000 to 2021-03-20 19:32:26.350000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-20 19:32:26.350000 to 2021-03-20 19:32:26.400000
>> new tag(s): ['x.t

...

>> [column definitions OK]
>> processing row: [last rows 8]>> processing row: [last rows 8]


>> [column definitions OK]>> [column definitions OK]
>> [column definitions OK]>> [column definitions OK]
>> [column definitions OK]
>> processing row: >> [column definitions OK]>> [column definitions OK]>> [column definitions OK]>> [column definitions OK][last rows 8]>> [column definitions OK]>> [column definitions OK]
>> processing row: [last rows 8]

>> [column definitions OK]>> [column definitions OK]>> [column definitions OK]
>> processing row: >> [column definitions OK]
>> processing row: [last rows 8]
>> [column definitions OK][last rows 8]
>> [column definitions OK]>> [column definitions OK]

Loading-Extract-Transfer to Hub done, status OK, #rows = 2


>> [column definitions OK]>> [column definitions OK]

>> [column definitions OK]
>> processing row: [last rows 8]>> [column definitions OK]
>> processing row: [last rows 8]>> [column definitions OK]

>> processing row: 
>> processing row


Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2




[last rows 8]


Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-E


Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extract-Transfer to Hub done, status OK, #rows = 2



Loading-Extract-Transfer to Hub done, status OK, #rows = 2




Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extract-Transfer to Hub done, status OK, #rows = 2





Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extract-Transfer to Hub done, status OK, #rows = 2









Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extract-Transfer to Hub done, status OK, #rows = 2




Loading-Extract-Transfer to Hub done, status OK, #rows = 2





Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extra


Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extract-Transfer to Hub done, status OK, #rows = 2



Loading-Extract-Transfer t



Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2



Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extract-Transfer to 